In [13]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

Files already downloaded and verified
Epoch [1/80], Step [100/500] Loss: 1.5430
Epoch [1/80], Step [200/500] Loss: 1.2956
Epoch [1/80], Step [300/500] Loss: 1.4688
Epoch [1/80], Step [400/500] Loss: 1.2070
Epoch [1/80], Step [500/500] Loss: 1.1124
Epoch [2/80], Step [100/500] Loss: 1.1737
Epoch [2/80], Step [200/500] Loss: 1.0844
Epoch [2/80], Step [300/500] Loss: 1.0846
Epoch [2/80], Step [400/500] Loss: 1.0322
Epoch [2/80], Step [500/500] Loss: 0.7611
Epoch [3/80], Step [100/500] Loss: 0.9780
Epoch [3/80], Step [200/500] Loss: 1.0442
Epoch [3/80], Step [300/500] Loss: 0.8265
Epoch [3/80], Step [400/500] Loss: 0.7712
Epoch [3/80], Step [500/500] Loss: 0.8189
Epoch [4/80], Step [100/500] Loss: 0.6783
Epoch [4/80], Step [200/500] Loss: 0.7233
Epoch [4/80], Step [300/500] Loss: 0.8422
Epoch [4/80], Step [400/500] Loss: 0.6118
Epoch [4/80], Step [500/500] Loss: 0.7332
Epoch [5/80], Step [100/500] Loss: 0.7075
Epoch [5/80], Step [200/500] Loss: 0.6907
Epoch [5/80], Step [300/500] Loss: 0.8

Epoch [39/80], Step [200/500] Loss: 0.1792
Epoch [39/80], Step [300/500] Loss: 0.1526
Epoch [39/80], Step [400/500] Loss: 0.1846
Epoch [39/80], Step [500/500] Loss: 0.2160
Epoch [40/80], Step [100/500] Loss: 0.1859
Epoch [40/80], Step [200/500] Loss: 0.1310
Epoch [40/80], Step [300/500] Loss: 0.2181
Epoch [40/80], Step [400/500] Loss: 0.2906
Epoch [40/80], Step [500/500] Loss: 0.2707
Epoch [41/80], Step [100/500] Loss: 0.1258
Epoch [41/80], Step [200/500] Loss: 0.1870
Epoch [41/80], Step [300/500] Loss: 0.1760
Epoch [41/80], Step [400/500] Loss: 0.1711
Epoch [41/80], Step [500/500] Loss: 0.2392
Epoch [42/80], Step [100/500] Loss: 0.1383
Epoch [42/80], Step [200/500] Loss: 0.1290
Epoch [42/80], Step [300/500] Loss: 0.1736
Epoch [42/80], Step [400/500] Loss: 0.2072
Epoch [42/80], Step [500/500] Loss: 0.2002
Epoch [43/80], Step [100/500] Loss: 0.2925
Epoch [43/80], Step [200/500] Loss: 0.1144
Epoch [43/80], Step [300/500] Loss: 0.1350
Epoch [43/80], Step [400/500] Loss: 0.1932
Epoch [43/8

Epoch [77/80], Step [300/500] Loss: 0.1217
Epoch [77/80], Step [400/500] Loss: 0.0850
Epoch [77/80], Step [500/500] Loss: 0.1063
Epoch [78/80], Step [100/500] Loss: 0.0603
Epoch [78/80], Step [200/500] Loss: 0.0894
Epoch [78/80], Step [300/500] Loss: 0.1098
Epoch [78/80], Step [400/500] Loss: 0.1228
Epoch [78/80], Step [500/500] Loss: 0.1214
Epoch [79/80], Step [100/500] Loss: 0.1191
Epoch [79/80], Step [200/500] Loss: 0.1198
Epoch [79/80], Step [300/500] Loss: 0.2401
Epoch [79/80], Step [400/500] Loss: 0.1724
Epoch [79/80], Step [500/500] Loss: 0.1865
Epoch [80/80], Step [100/500] Loss: 0.0954
Epoch [80/80], Step [200/500] Loss: 0.1384
Epoch [80/80], Step [300/500] Loss: 0.1155
Epoch [80/80], Step [400/500] Loss: 0.1948
Epoch [80/80], Step [500/500] Loss: 0.1707
Accuracy of the model on the test images: 88.25 %
